In [2]:
%pwd

'/Users/enitan/Desktop/Projects/Chest-Disease-Classification-MLOPs/research'

In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
from chest_disease_classification import logger
from chest_disease_classification.utils import read_yaml, create_directory
import gdown
import zipfile


In [6]:
# class ConfigurationManager:
#     def __init__(self, config_file_path, params_file_path):
#         self.config_file_path = config_file_path
#         self.params_file_path = params_file_path

#         self.config = read_yaml(self.config_file_path)
#         self.params = read_yaml(self.params_file_path)

#         create_directory(self.config.artifact_root)

#     def get_data_ingestion_config(self) -> DataIngestionConfig:
#         config_data_ingestion = self.config.data_ingestion
#         create_directory(config_data_ingestion.root_dir)

#         data_ingestion_config = DataIngestionConfig(
#         root_dir = config_data_ingestion.root_dir,
#         source_url = config_data_ingestion.source_url,
#         local_data_file = config_data_ingestion.local_data_file,
#         unzip_dir = config_data_ingestion.unzip_dir
#         )
#         return data_ingestion_config



In [7]:
# class DataIngestion:
#     def __init__(self, config: DataIngestionConfig):
#         self.config = config

#     def download_data(self):
#         try:
#             data_url = self.config.source_url
#             unzip_url = self.config.unzip_dir
#             local_data_file = self.config.local_data_file

#             os.makedirs(unzip_url, exist_ok = True)
#             file_id = data_url.split("/")[-2]
#             prefix = "https://drive.google.com/uc?/export=download&id="
#             logger.info(f"Downloading data from {data_url} to {unzip_url}")
#             gdown.download(prefix+file_id, local_data_file)
#             logger.info("file downloaded successfully")
#         except Exception as e:
#             raise e

#     def unzip_file(self):

#         try:
#             unzip_file_path = self.config.unzip_dir
#             data_path = self.config.local_data_file
#             os.makedirs(unzip_file_path, exist_ok = True)
#             logger.info("Unzipping the project data into {unzip_file_path}")
#             with zipfile.ZipFile(data_path, "r") as file:
#                 file.extractall(unzip_file_path)
#             logger.info("Project data successfully unzip at {unzip_file_path}")
#         except Exception as e:
#             raise e

        
        


In [8]:
# try:
#     logger.info("initializing data ingestion component")
#     config = ConfigurationManager(config_file_path = Path("config/config.yaml"), params_file_path = Path("params.yaml"))
#     data_ingestion_config = config.get_data_ingestion_config()
#     data_ingestion = DataIngestion(config = data_ingestion_config)
#     data_ingestion.download_data()
#     data_ingestion.unzip_file()
# except Exception as e:
#     raise e


In [9]:
from pathlib import Path
from dataclasses import dataclass

In [10]:
@dataclass
class BaseModelConfig:
    root_dir: Path
    base_model_dir: Path
    updated_base_model_dir: Path
    params_include_top: bool
    params_image_size: list
    params_classes: int
    params_weights: str
    params_learning_rate: float

In [11]:
class ConfigurationManager:
    def __init__(self, config_file_path, params_file_path):
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path

        self.config = read_yaml(self.config_file_path)
        self.params = read_yaml(self.params_file_path)

        create_directory(self.config.artifact_root)

    def get_base_model_config(self) -> BaseModelConfig:
        config_base_model = self.config.basemodel
        create_directory(config_base_model.root_dir)

        base_model_config = BaseModelConfig(
            root_dir = Path(config_base_model.root_dir),
            base_model_dir = Path(config_base_model.base_model),
            updated_base_model_dir = Path(config_base_model.update_base_model),
            include_top = self.params.include_top,
            image_size = self.params.image_size,
            classes = self.params.classes,
            weights = self.params.weights,
            learning_rate = self.params.learning_rate
        )
        return base_model_config



In [12]:
# import tensorflow as tf

In [13]:
class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config

    @staticmethod
    def save_model(path:Path, model = tf.keras.Model):
        model.save(path)

    def get_base_model(self):
        self.model = keras.applications.VGG16(
                        include_top = self.config.include_top,
                        weights = self.config.weights,
                        classes = self.config.classes,
                    )
        self.save_model(path = self.config.base_model_dir, model = self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learing_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not none) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = "softmax",
            )(flatten_in)

        full_model = tf.keras.models.Model(
            input = model.input,
            output = prediction
        )

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = metric["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        full_model = self.prepare_full_model(
            model = self.model,
            classes = self.config.classes, 
            freeze_all = True, 
            freeze_till = None, 
            learing_rate = self.config.learning_rate
            )
        
        self.save_model(path = self.config.updated_base_model_dir, model = full_model)


        